# 개요
- 교과목: 빅데이터 시스템 개발
- 능력단위: 빅데이터 수집시스템 개발
- 훈련교사명: 정지훈 강사
- 수강생명: 전우정
- 평가방법: 기타(작업형)
- 평가예정일: 2022년 01월 20일 목요일 (1시간)

# 출제 문항
- 네이버 뉴스 데이터의 댓글을 수집하는 코드를 완성하세요. 
  + 뉴스기사: https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=009&aid=0004909400


# 답안 코드

In [3]:
# 크롤링 라이브러리
from bs4 import BeautifulSoup
import requests
import re

# 데이터프레임
import pandas as pd

# 샘플 URL을 적용한다. 
# url = "https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=009&aid=0004909400" 
url = "https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=001&aid=0012924368&isYeonhapFlash=Y&rc=N"

def get_df(url):
  # 댓글을 달 빈 리스트를 생성합니다.
  List = []
  url = url
  oid = url.split("oid=")[1].split("&")[0]
  aid = url.split("aid=")[1]
  page = 1
  header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": url,

  }
  while True:
    c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
          page) + "&refresh=false&sort=FAVORITE"
    # 파싱하는 단계입니다.
    req = requests.get(c_url, headers=header)
    print("상태코드", req.status_code)
    cont = BeautifulSoup(req.content, "html.parser")
    total_comm = str(cont).split('comment":')[1].split(",")[0]
    print("전체 댓글 개수: ", total_comm)

    match = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
    # 댓글을 리스트에 중첩합니다.
    List.append(match)

    # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
    if int(total_comm) <= ((page) * 20):
      break
    else:
      page += 1

		# 
  def flatten(l):
    flatList = []
    for elem in l:
      if type(elem) == list:
        for e in elem:
          flatList.append(e)
      else:
        flatList.append(elem)
    return flatList

    # 리스트 결과입니다.
    # print(flatten(List))

    # convert dataframe
  data = pd.DataFrame(flatten(List), columns=["기사댓글"])
  data = data.rename_axis("index").reset_index()

  # write_excel
  # data.to_excel("news_comments.xlsx", sheet_name="Sheet1")
  return data

data = get_df(url) # URL 테스트 시, 실행
data.to_excel("20220120_홍길동.xlsx", sheet_name="Sheet1")
data

상태코드 200
전체 댓글 개수:  116
상태코드 200
전체 댓글 개수:  116
상태코드 200
전체 댓글 개수:  116
상태코드 200
전체 댓글 개수:  117
상태코드 200
전체 댓글 개수:  117
상태코드 200
전체 댓글 개수:  117


,index,기사댓글
0,0,"""누가보면 미접종자가 코로나 다 옮긴줄알겠네 ㅉㅉ"""
1,1,"""이딴 놈들이 전 정권 메르스&amp; 세월호 사건 때 대처 못했다고 입 털었 ㅋㅋ..."
2,2,"""현직의사 백신실체 독극물 이물질 괴생명체 다 밝혀냈다.(검색. 네이버-백신실체-동..."
3,3,"""법원 판단도 '유감입니다' 이러면서 그래도 우리는 청소년 방역패스 할겁니다. 라고..."
4,4,"""할거면 전부 다 풀어라!!!"""
...,...,...
111,111,"""미접종자는 완전방어해야한다 미접종자는 치료비도 자가부담시켜야한다"""
112,112,"""다풀고 확산되면 풀었다고 난리칠 인간들, 불과 일년전 백신 안들여 온다고 개..."
113,113,"""아이들 많이 다니는 번화가 가서 10분만 서있어봐라. 왜 청소년 방역패스가 꼭 필..."
114,114,"""자 서울 백화점 마트에서 대규모 집단 감염 한번가자~~~ 그리고 사직서 낸 판사 ..."


In [ ]:
#